# Курс Data Science
## Модуль 11.4 Метрики качества линейной регрессии

В задачах машинного обучения мы хотим сравнивать несколько моделей машинного обучения и выбирать наилучшую из них. Решение о том, какая модель хорошая, а какая плохая, принимается на основе одной или нескольких *метрик* моделей машинного обучения.

Без метрик обучение моделей вообще теряет всякий смысл – как же определить, какая из зоопарка обученных моделек хорошая, а какая плохая? Давайте разберёмся, как определить лучшую модель с помощью математики

Давайте вернёмся к картинке из первого урока. От каждой красной точки до синей линии отложены отрезки, которые представляют собой ошибку регрессии – как сильно модель ошибается на каждой из точек.

![least_squares_learning](https://248006.selcdn.ru/public/Data-science-4/img/least_line.png)

Интуиция за метриками стоит очень простая – давайте как-нибудь усредним отклонения по всем точкам и получим одно число – метрику качества линейной регрессии, т.е. насколько модель отклоняется от реальных данных.

Метрики принимают на вход два вектора, предсказания модели и истинные значения, после чего вычисляют по этим векторам качество модели.

Сначала загрузим данные эксперимента, датасет с ценами на дома в Бостоне

In [10]:
from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
features = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

reg = LinearRegression().fit(features, y)

Теперь получим два вектора – предказанное значение $\hat{y}$ и  истинное значение $y$:

In [11]:
y_pred = reg.predict(features) # предсказанное значение
y_true = y # истинное значение

Теперь посмотрим, какие функции можно применять к этим двум наборам точек

## Mean absolute error

Для оценки качества регрессии можно использовать среднюю абсолютную ошибку

In [17]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(reg.predict(features), y)
print(f"MAE = {mae}")

MAE = 3.270862810900316


*Mean Absolute Error* - это просто сумма отклонений истинных значений $y$ от предсказаний нашей модели:

$$
\text{absolute error} = |y_1 - \hat{y}_1| + |y_2 - \hat{y}_2| + \ldots
$$

А потом мы эту сумму делим на количество точек - получаем среднюю ошибку

Метрика принимает только положительные значения! Чем ближе к нулю, тем лучше модель.

## MSE

Mean Squared Error (MSE) - это базовая метрика для определения качества линейной регрессии

Для каждого предсказанного значения $\hat{y}_i$ мы считаем квадрат отклонения от фактического значения и считаем среднее по полученным величинам

In [19]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_true, y_pred)
print(f"MSE = {mse}")

MSE = 21.894831181729202


В целом логика та же, что в *MAE*, только усреднять мы будем квадраты ошибок 
$$
\text{absolute error} = (y_1 - \hat{y}_1)^2 + (y_2 - \hat{y}_2)^2 + \ldots
$$

Эта метрика тоже принимает только положительные значения! Чем ближе к нулю, тем лучше модель.

Эта ошибка визуально похожа на  *MSE*, но на графике видно, что *MAE* (красная линия) почти всегда меньше по значению, чем MSE (синяя линия). Это значит, что *MSE* более "пессимистична" и сильнее штрафует за большие ошибки - т.е. MSE лучше применять, когда вы уверены что в выборке нет "выборосов" (англ. outliers) - значений, который очень сильно отличаются от остальных точек. В этом случае MSE может быть очень плохой, а на деле ситуация приемлема. Если выбросы есть, лучше применять MAE.

![rmse_vs_mae](https://248006.selcdn.ru/public/Data-science-4/img/rmse_vs_mae.png)

## $R^2$ (коэффициент детерминации)

Название - калька английского термина *coefficient of determination*. Наилучшее возможное значение 1.0, чем меньше тем хуже. Если этот коэффициент близок к 1, то условная дисперсия модели (то есть разброс предсказаний модели $\hat{y}$ относительно разброса самой целевой переменной $y$ ) достаточно мала - то есть модель неплохо описывает данные. Коэффициент может быть даже отрицательным - то это значит, что модель совсем уж плохая.

Эта метрика хороша тем, что она *нормализована*, то есть не превышает единицу - удобно сравнивать разные модели. Например, метрика $MSE$ может принимать ничем не ограниченные значения больше нуля - это не всегда удобно.

В библиотеке *sklearn* есть готовая реализация этой метрики.

In [20]:
from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print(f"r2_score = {r2}")

r2_score = 0.7406426641094095


Про другие ошибки можно почитать в [официальной документации](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics) в разделе про метрики регрессии.